# Complete documentation of reexcution of TWAS Rosmap MV Susie analysis
Plan:
1. Estimation of whole genome uni susie
2. Prior estimation with the new flashier pipeline
2.5 estimation of GCTA heritability for whole genome
2.5 reestimation of GCTA heritability
3. whole genome merging, remove extracted snps


# Data reformatting

In [ ]:
sos run ~/GIT/neuro-twas/Workflow/SOS_ROSMAP_gene_exp_processing.ipynb \
    --ref "/home/hs3163/Project/Rosmap/data/patient_key_WGS.txt" \
    --name_col 2 \
    --real_name_col 1 \
    --gene_exp "/mnt/mfs/ctcn/datasets/rosmap/rnaseq/dlpfcTissue/batch0-m5/values/residuals/geneTpmResidualsAgeGenderAdj.txt" \
    --start_at 2 \
    --output "/home/hs3163/Project/Rosmap/data/gene_exp/DLPFC" \
    -J 6 -q csg -c ~/system_file/csg.yml

sos run ~/GIT/neuro-twas/Workflow/SOS_ROSMAP_gene_exp_processing.ipynb \
    --ref "/home/hs3163/Project/Rosmap/data/patient_key_WGS.txt" \
    --name_col 2 \
    --real_name_col 1 \
    --gene_exp "/mnt/mfs/ctcn/datasets/rosmap/rnaseq/pccTissue/mbatch1-5/values/residuals/geneTpmResidualsAgeGenderAdj.txt" \
    --start_at 2 \
    --output "/home/hs3163/Project/Rosmap/data/gene_exp/PCC" \
    -J 6 -q csg -c ~/system_file/csg.yml

sos run ~/GIT/neuro-twas/Workflow/SOS_ROSMAP_gene_exp_processing.ipynb \
    --ref "/home/hs3163/Project/Rosmap/data/patient_key_WGS.txt" \
    --name_col 2 \
    --real_name_col 1 \
    --gene_exp "/mnt/mfs/ctcn/datasets/rosmap/rnaseq/acTissue/mbatch1-4/values/residuals/geneTpmResidualsAgeGenderAdj.txt" \
    --start_at 2 \
    --output "/home/hs3163/Project/Rosmap/data/gene_exp/AC" &

## Generation of Whole genome prior
with modified windows and shared SNPs

In [ ]:
nohup sos run ~/GIT/neuro-twas/Workflow/susie-wgs-prior.ipynb susie  \
  --molecular-pheno  /home/hs3163/Project/Rosmap/data/gene_exp/AC/geneTpmResidualsAgeGenderAdj_rename.txt  \
  --wd /home/hs3163/Project/Genome_prior/AC \
  --genotype_list /home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt \
  --region_list /home/hs3163/Project/Rosmap/data/gene_exp/AC/geneTpmResidualsAgeGenderAdj_rename_region_list.txt \
  --region_name 1 \
  --data_start 2 \
  --window 500000 \
  --container /mnt/mfs/statgen/containers/twas_latest.sif \
  --job_size 1\
  -J 50 -q csg -c /mnt/mfs/statgen/pbs_template/csg.yml -s build &

In [ ]:
nohup sos run ~/GIT/neuro-twas/Workflow/susie-wgs-prior.ipynb susie  \
  --molecular-pheno  /home/hs3163/Project/Rosmap/data/gene_exp/PCC/geneTpmResidualsAgeGenderAdj_rename.txt  \
  --wd /home/hs3163/Project/Genome_prior/PCC \
  --genotype_list /home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt \
  --region_list /home/hs3163/Project/Rosmap/data/gene_exp/PCC/geneTpmResidualsAgeGenderAdj_rename_region_list.txt \
  --region_name 1 \
  --data_start 2 \
  --window 500000 \
  --container /mnt/mfs/statgen/containers/twas_latest.sif \
  --job_size 1\
  -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

In [ ]:
nohup sos run ~/GIT/neuro-twas/Workflow/susie-wgs-prior.ipynb susie  \
  --molecular-pheno  /home/hs3163/Project/Rosmap/data/gene_exp/DLPFC/geneTpmResidualsAgeGenderAdj_rename.txt  \
  --wd /home/hs3163/Project/Genome_prior/DLPFC \
  --genotype_list /home/hs3163/Project/Rosmap/data/Rosmap_wgs_genotype_list.txt \
  --region_list /home/hs3163/Project/Rosmap/data/gene_exp/DLPFC/geneTpmResidualsAgeGenderAdj_rename_region_list.txt \
  --region_name 1 \
  --data_start 2 \
  --window 500000 \
  --container /mnt/mfs/statgen/containers/twas_latest.sif \
  --job_size 1\
  -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

# Compute the mixture prior ( Use the updated version)

## Merging the tissues

In [ ]:
nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb convert_plink \
--molecular_pheno /home/hs3163/Project/Genome_prior/data/molc_dir    \
--analysis-units /home/hs3163/Project/Genome_prior/data/rds_list_test  \
--cwd   /home/hs3163/Project/Genome_prior/mixture_prior/output \
--name "geneTpmResidualsAgeGenderAdj_rename" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
-s build &

## Extracting the strongest, null .etc signals

In [ ]:
    cd /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/  && ls *.rds | sed 's/\.rds//g' > analysis_units.txt && cd -
    nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb extract_effects \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ &

## Create flashr output


In [ ]:
nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb flash \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &
        
nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb flash_nonneg \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb pca \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &
        
nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb canonical \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &


## Extreme Deconvolution

In [ ]:
for m in /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/; do
    sos run mixture_prior.ipynb ud \
        --datadir $m --name `basename $m` &> ed_$m.log
    sos run mixture_prior.ipynb ud --ud-method teem \
        --datadir $m --name `basename $m` &> teem_$m.log
    sos run mixture_prior.ipynb ed \
        --datadir $m --name `basename $m` &> bovy_$m.log
done

nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb ud \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb ud --ud-method teem  \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &
        
nohup sos run ~/GIT/bioworkflows/multivariate-fine-mapping/mixture_prior.ipynb ed \
        --analysis-units /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/analysis_units.txt \
        --datadir /home/hs3163/Project/Genome_prior/mixture_prior/output/RDS/ \
        --name 'susie'  \
        --container /mnt/mfs/statgen/containers/twas_latest.sif \
        --cwd /home/hs3163/Project/Genome_prior/mixture_prior/output/ \
        -J 50 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &



# MV SuSie with mixture prior

In [ ]:
nohup sos run ~/GIT/neuro-twas/Workflow/mv_susie.ipynb mv_susie \
--molecular_pheno_dir "/mnt/mfs/statgen/neuro-twas/mv_wg/molecular_phenotype_list" \
--region_list /mnt/mfs/statgen/neuro-twas/mv_wg/wg_rds_list_final \
--wd "/mnt/mfs/statgen/neuro-twas/whole_genome_0504/" \
--name_prefix "geneTpmResidualsAgeGenderAdj_rename" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
--impute TRUE --job_size 1 \
--cv_times 100 \
--mixture_prior '/home/hs3163/Project/Genome_prior/mixture_prior/output/susie.ed.rds' \
-J 200 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

nohup sos run ~/GIT/neuro-twas/Workflow/mv_susie.ipynb mv_susie \
--molecular_pheno_dir "/mnt/mfs/statgen/neuro-twas/mv_wg/molecular_phenotype_list" \
--region_list /mnt/mfs/statgen/neuro-twas/mv_wg/wg_rds_list_final \
--wd "/mnt/mfs/statgen/neuro-twas/whole_genome_0504/" \
--name_prefix "geneTpmResidualsAgeGenderAdj_rename" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
--impute TRUE --job_size 1 \
--cv_times 100 \
--mixture_prior '/home/hs3163/Project/Genome_prior/mixture_prior/output/susie.ed_bovy.rds' \
-J 200 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &

nohup sos run ~/GIT/neuro-twas/Workflow/mv_susie.ipynb mv_susie \
--molecular_pheno_dir "/mnt/mfs/statgen/neuro-twas/mv_wg/molecular_phenotype_list" \
--region_list /mnt/mfs/statgen/neuro-twas/mv_wg/wg_rds_list_final \
--wd "/mnt/mfs/statgen/neuro-twas/whole_genome_0504/" \
--name_prefix "geneTpmResidualsAgeGenderAdj_rename" \
--container /mnt/mfs/statgen/containers/twas_latest.sif \
--impute TRUE --job_size 1 \
--cv_times 100 \
--mixture_prior '/home/hs3163/Project/Genome_prior/mixture_prior/output/susie.teem.rds' \
-J 200 -q csg -c ~/GIT/neuro-twas/template/csg.yml -s build &